# 8. Merge the walking network and bike network

In [ ]:
# Standard library and path imports
import set_path
import warnings

# Third-party library imports
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely.geometry as sg
import networkx as nx
import folium
import momepy
from shapely import wkt
from tqdm.notebook import tqdm

# Local or project-specific imports
import plot_utils
import crossing_utils
import curb_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

## Import walking network, bike network and curb heights

In [ ]:
# Import walking, road and bike network
gdf_walking_network = gpd.read_file(cf.output_file_widths).to_crs(crs=st.CRS)
gdf_walking_nodes = gpd.GeoDataFrame(geometry = gdf_walking_network.boundary.explode(index_parts=True), crs=st.CRS)
gdf_walking_nodes['x'], gdf_walking_nodes['y'] = gdf_walking_nodes.geometry.x, gdf_walking_nodes.geometry.y
gdf_road_network = gpd.read_file(cf.output_road_network).to_crs(crs=st.CRS)
gdf_bike_network = gpd.read_file(cf.output_bike_network).to_crs(crs=st.CRS)
gdf_bike_path_polygons = gpd.read_file(cf.output_bikepaths_bike_network).to_crs(crs=st.CRS)

# Import curb heights
gdf_ch = gpd.read_file(cf.output_curb_heigts)

In [ ]:
# Cut bike network in pieces of length distance_delta and obtain corresponding node coordinates
distance_delta = 10
df_bike_network_cut = pd.DataFrame(columns=gdf_bike_network.columns)
geoms = []
for i, row in gdf_bike_network.iterrows():
    points_on_line = curb_utils.get_points_on_line(row['geometry'], distance_delta)
    result = curb_utils.split_line_by_point(row['geometry'], points_on_line)
    for line_segm in result:
        df_bike_network_cut.loc[len(df_bike_network_cut)] = row
        geoms.append(line_segm)

gdf_bike_network_cut = gpd.GeoDataFrame(df_bike_network_cut, geometry=geoms, crs=st.CRS)
gdf_bike_network_cut['length'] = gdf_bike_network_cut['geometry'].length
gdf_bike_network_cut =  gdf_bike_network_cut.drop(columns=['index'])
gdf_bike_nodes = gpd.GeoDataFrame(geometry = gdf_bike_network_cut.boundary.explode(index_parts=True), crs=st.CRS)
gdf_bike_nodes['x'], gdf_bike_nodes['y'] = gdf_bike_nodes.geometry.x, gdf_bike_nodes.geometry.y

In [ ]:
# Calculate connected components for pedestrian network
graph_network = momepy.gdf_to_nx(gdf_walking_network, approach='primal')
cc_network = list(nx.connected_components(graph_network))
nodes_in_edge = [line.coords[0] for line in gdf_walking_network['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_network)) if node in cc_network[i]]
gdf_walking_nodes['cc'] = [x for y in zip(cc_ids, cc_ids) for x in y]

## Generate edges to connect walk and bike network

### Set max switch height (curb height) to switch between the networks
In practice, settings a maximum curb height up to approximately 0.04m equals a smooth transition between the networks. This is due to noise in the point cloud. \
This parameter increments linearly, so setting max_height at 0.06m would roughly equal a max curb height of 0.02m etc. 

In [ ]:
# SET THIS VALUE
max_network_switch_height = 0.04 # in meter

gdf_ch.dropna(subset=['curb_height'])
gdf_ch = gdf_ch.loc[(gdf_ch['curb_height'] <= max_network_switch_height)]
gdf_ch['centroid'] = gdf_ch['geometry'].centroid
gdf_ch.reset_index(drop=True, inplace=True)

### Only keep curbs which connect bike path and sidewalk

In [ ]:
idxs = [i for i, row in gdf_ch.iterrows() if len(gdf_bike_path_polygons.intersects(row['geometry']).unique())  > 1]
switch_curbs = gdf_ch.loc[idxs]

### Obtain helper edges from curb to walking network and from curb to cycle network

In [ ]:
# Get connections
gdf_ch['geometry'] = gdf_ch['centroid']
gdf_ch['x'], gdf_ch['y'] = gdf_ch.geometry.x, gdf_ch.geometry.y

walking_edges, walking_edges_geometries = crossing_utils.get_connections(gdf_ch, gdf_walking_nodes, max_dist=5, max_connections=1, 
                                                                         include_cc_rule=True, cc_column='cc', return_gdf=False)
cycling_edges, cycling_edges_geometries = crossing_utils.get_connections(gdf_ch, gdf_bike_nodes, max_dist=5, max_connections=3, return_gdf=False)

### Obtain connections edges from walking network to cycling network without curb node

In [ ]:
connection_edges, walk_nodes, cycle_nodes = [], [], []
for i in tqdm(range(len(walking_edges))):
    walking_edge = walking_edges[i]
    for cycling_edge in cycling_edges:
        if walking_edge[0] == cycling_edge[0]: # check if edges share curb node
            walk_node = gdf_walking_nodes.iloc[walking_edge[1]]['geometry']
            cycle_node = gdf_bike_nodes.iloc[cycling_edge[1]]['geometry']
            con_edge = sg.LineString([walk_node, cycle_node])
            walk_nodes.append(walk_node.coords[0])
            cycle_nodes.append(cycle_node.coords[0])
            connection_edges.append(con_edge)
            
gdf_connection_edges = gpd.GeoDataFrame(geometry=np.asarray(connection_edges, dtype="object"), crs=st.CRS)
gdf_connection_edges['walk_node'] = walk_nodes
gdf_connection_edges['cycle_node'] = cycle_nodes
gdf_connection_edges['length'] = gdf_connection_edges['geometry'].length

### Remove duplicate and unnecessary edges

In [ ]:
# Determine connected component walking nodes and remove duplciate edges
for i, row in gdf_connection_edges.iterrows():
    for j in range(len(cc_network)):
        if row['walk_node'] in cc_network[j]:
            gdf_connection_edges.loc[i, 'walk_cc'] = j

# Remove duplicate edges          
gdf_connection_edges = gdf_connection_edges.groupby(['cycle_node', 'walk_node']).nth(0)

# If cycle node is connected to multiple walk nodes that belong to same connected compenent, only keep shortest connection
gdf_connection_edges = gdf_connection_edges.sort_values('length', ascending=True).groupby(['cycle_node', 'walk_cc']).nth(0)

# Delete unecessary columns
gdf_connection_edges.reset_index(inplace=True)
gdf_connection_edges = gdf_connection_edges.drop(columns=['index', 'walk_node', 'cycle_node', 'walk_cc', 'length'])

# # Remove connections that are too long
gdf_connection_edges = gdf_connection_edges.loc[gdf_connection_edges.geometry.length < 6]

## Store

In [ ]:
# Save connection edges as geopackage
gdf_connection_edges.to_file(cf.output_walk_bike_connections_base + '_max_height_{}.gpkg'.format(max_network_switch_height), driver='GPKG')

# Save splitted bike network
gdf_bike_network_cut.to_file(cf.output_bike_network_cut, driver='GPKG')

## Visualize connections

In [ ]:
# set True for satellite background, False for regular background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.7940192423718443], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network and new edges
geo_j = folium.GeoJson(gdf_walking_network, style_function=lambda x: {"color": "black", "weight": 3}).add_to(map)
geo_j = folium.GeoJson(gdf_bike_network, style_function=lambda x: {"color": "green", "weight": 3}).add_to(map)
geo_j = folium.GeoJson(gdf_connection_edges, style_function=lambda x: {"color": "red", "weight": 3}).add_to(map)

# plot map
map